In [1]:
from ipywidgets import FileUpload, Button, Output, Dropdown, HBox, VBox
from IPython.display import display
import torch
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel, DDIMScheduler
from PIL import Image
import cv2
import numpy as np
import io
from tqdm import tqdm

# Create a file upload widget to allow the user to upload an image
image_upload = FileUpload(description="Upload an image:", multiple=False)

# Create a pull-down menu of style transfer options
style_options = [
    "Van Gogh",
    "Monet",
    "Picasso",
    "Warhol",
    "Pollock",
    "Kandinsky",
    "Matisse",
    "Cezanne",
    "Gauguin",
    "Seurat",
    "Renoir",
    "Degas",
    "Toulouse-Lautrec",
    "Goya",
    "El Greco",
    "Rembrandt",
    "Vermeer",
    "Caravaggio",
    "Botticelli",
    "Michelangelo"
]
style_dropdown = Dropdown(options=style_options, value=style_options[0], description="Style:")

# Create a button to generate the image
generate_button = Button(description="Generate Image")

# Create an output widget to display the generated image
output = Output()

# Define a function to generate the image when the button is clicked
def generate_image(b):
    with output:
        output.clear_output(wait=True)
        # Get the uploaded image from the file upload widget
        image_file = image_upload.value[0]
        image_bytes = image_file["content"]
        image = Image.open(io.BytesIO(image_bytes))
        image = image.resize((512, 512))
        display(image)

        # Convert the image to a numpy array and apply Canny edge detection
        image_array = np.array(image)
        low_threshold = 100
        high_threshold = 200
        image_array = cv2.Canny(image_array, low_threshold, high_threshold)
        image_array = image_array[:, :, None]
        image_array = np.concatenate([image_array, image_array, image_array], axis=2)
        canny_image = Image.fromarray(image_array)

        # Get the style from the pull-down menu
        style = style_dropdown.value

        # Load the Stable Diffusion model and control net
        controlnet = ControlNetModel.from_pretrained("lllyasviel/sd-controlnet-canny", torch_dtype=torch.float16)
        pipe = StableDiffusionControlNetPipeline.from_pretrained(
            "runwayml/stable-diffusion-v1-5", controlnet=controlnet, torch_dtype=torch.float16
        )
        pipe.scheduler = DDIMScheduler.from_config(pipe.scheduler.config)
        pipe = pipe.to("xpu")

        # Generate the image using the Stable Diffusion pipeline
        with output:
            output.clear_output(wait=True)
            pbar = tqdm(total=1, desc="Generating image")
            with torch.inference_mode():
                output_image = pipe(
                    style, image=canny_image
                ).images[0]
            pbar.update(1)
            pbar.close()
            display(output_image)

            # Write the generated image to a PNG file
            random_number = random.randint(1, 999)
            output_image.save("generated_image.png")

# Link the button to the generate_image function
generate_button.on_click(generate_image)

# Display the widgets in the notebook
display(VBox([image_upload, style_dropdown, generate_button, output]))